## How to use user_classification.py

<hr>

First import the module and initialize a twitter api if needed.

In [1]:
from user_classification import *

In [2]:
import twitter

CONSUMER_KEY = 'oEWIMZpIUQCULqXLVAydLsUoT'
CONSUMER_SECRET = 'F31ZPD7rRlzkrsbmE5JqMs8tWQjLrD3TxkNObNNeHz2nBbDwnt'
OAUTH_TOKEN = '1303957818169585664-5xyRbIo7kcM19IQjT7t2OdtBS5G0hw'
OAUTH_TOKEN_SECRET = 'WKZNyiVxqMyfxL3JoHiaU30axOrpGu7YasKjtmYhnz2Ym'

auth = twitter.oauth.OAuth(OAUTH_TOKEN,
                          OAUTH_TOKEN_SECRET, 
                          CONSUMER_KEY, 
                          CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

Now we will initialize a dataset object.

If a new dataset is needed, we will need a csv with labelled data. It must have at least two columns:
- `screen_name`: column with user accounts.
- `label`: column with the user label.

If we want to load an old dataset, we will make use of the `load` method.

In [3]:
dataset = Dataset()

In [4]:
new_dataset = False
csv_path = 'Data/labelled_users.csv'
user_column = 'screen_name'
label_column = 'clas_1'

def lowercase(x):
    return x.lower()

if new_dataset:
    df = pd.read_csv(csv_path)
    user = list(df[user_column].to_numpy())
    label = list(df[label_column].to_numpy())
    label = [lowercase(x) for x in label]
    dataset.add_list_user(twitter_api, user, label)

In [5]:
old_dataset_path = 'Data/test.csv'

if not new_dataset:
    dataset.load(path=old_dataset_path)

We can check the dataset by printing the dataframe.

In [6]:
dataset.df.head()

,collective_activeness,collective_influence,degree_inclination,fav_count,label,life_time,mentions,plain_statuses,promotion_score,replies_given,retweets,std_hashtags,std_url,total_hashtag,total_url,user,verified
0,497.6,651.0,4.8,573.0,personal,9.188227,14.0,0.0,10.0,2.0,6.0,0.674949,0.421637,3.0,2.0,RepsolJackLeeds,0.0
1,1080.9,6275.0,1.8,5423.0,personal,2.617385,2.0,0.0,17.0,0.0,1.0,2.108185,0.316228,40.0,1.0,artemania2018,0.0
2,4866.6,49343.0,0.0,2278.0,corporative,9.577002,9.0,0.0,17.0,0.0,0.0,0.843274,0.483046,4.0,3.0,Repsol,1.0
3,3287.2,1197.0,4.2,902.0,personal,10.830938,5.0,0.0,23.0,1.0,3.0,1.763834,0.516398,10.0,4.0,JuanBellas,0.0
4,943.7,324.0,3.2,19.0,personal,9.979466,10.0,0.0,13.0,0.0,8.0,0.843274,0.000000,6.0,0.0,tarteka,0.0


We can save the dataset by calling to the `save` method.

In [7]:
save = False
dataset_path = 'Data/test.csv'

if save:
    dataset.save(path=dataset_path)

Now we will get the training data and we must specify if we want one-hot encoding (for multi-class classification) or not (for binary classification) by passing `to_categorical` variable.

In [8]:
to_categorical = False

data = dataset.get_training_data(test_size = 0.2, 
                                 to_categorical=to_categorical)

Now we can initialize a model by specifying the `model_type`. Then, we can train the model by providing the `data` object and calling the `train` method.

In [9]:
possible_type = ['random_forest',
                 'mlp',
                 'mlp_binary']

model_type = possible_type[2]

model = Model(model_type=model_type)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2048      
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 18,689
Trainable params: 18,689
Non-trainable params: 0
____________________________________________________

In [10]:
n_epochs = 12

model.train(data, n_epochs=n_epochs)

Epoch 1/12
60/60 [==============================] - 0s 4ms/step - loss: 0.5942 - accuracy: 0.6833
Epoch 2/12
60/60 [==============================] - 0s 4ms/step - loss: 0.4656 - accuracy: 0.8333
Epoch 3/12
60/60 [==============================] - 0s 4ms/step - loss: 0.3815 - accuracy: 0.8500
Epoch 4/12
60/60 [==============================] - 0s 6ms/step - loss: 0.3557 - accuracy: 0.8417
Epoch 5/12
60/60 [==============================] - 0s 4ms/step - loss: 0.3230 - accuracy: 0.8667
Epoch 6/12
60/60 [==============================] - 0s 5ms/step - loss: 0.3046 - accuracy: 0.8417
Epoch 7/12
60/60 [==============================] - 0s 5ms/step - loss: 0.3211 - accuracy: 0.8750
Epoch 8/12
60/60 [==============================] - 0s 5ms/step - loss: 0.2784 - accuracy: 0.8917
Epoch 9/12
60/60 [==============================] - 0s 5ms/step - loss: 0.2780 - accuracy: 0.9000
Epoch 10/12
60/60 [==============================] - 0s 5ms/step - loss: 0.2892 - accuracy: 0.8833
Epoch 11/12
60/60 [

In [11]:
model.evaluate(data) # get the test_set accuracy

15/15 [==============================] - 0s 5ms/step - loss: 0.2228 - accuracy: 0.8667


[0.22284862399101257, 0.8666666746139526]

We can save the model for latter use by providing a `name`. The model will be saved to checkpoints/`<model_type>`/`<name> (w/o extension)`/`<name>`.

In [12]:
name = '16_09_v1.h5'

model.save(name=name)

In [13]:
print(model.__doc__)


    Class to build, train and use models for classification. There are different types of models:
    ['random_forest', 'mlp', 'mlp_binary']

    ...

    Methods
    -------
    build:
        build the model depending on the model_type choosen.
    train: (data: data object)
        trains the model with the given data.
    evaluate: (data: data object)
        returns an evaluation of the model given a test data.
    predict: (x: iterable - np.array)
        returns the prediction for new data.
    save: (path: str)
        saves the model for later use.
    load: (path: str)
        loads a pretrained model.
    


In [14]:
print(dataset.__doc__)


    Class to handle the dataset and obtain the data from the twitter API.

    ...

    Methods
    -------
    load: (path: str)
        Loads an existing dataset saved in a csv.
    add_list_user: (twitter_api: twitter object, users: list, labels: list, force: bool)
        Add users from a list of screen_name (unique for Twitter) and, if needed, with the corresponding
        label for each user.
    add_user: (twitter_api: twitter object, users: list, labels: list, force: bool)
        Add users from a list of screen_name (unique for Twitter) and, if needed, with the corresponding
        label for each user.
    get_training_data: (columns: list, test_size: int)
        Returns a data object with the training and test data.
    get_data: (columns: list)
        Returns a np.array with the columns specified from the df.
    save: (path: str)
        Saves the dataframe into a csv file to be used latter. If the df was previously loaded,
        it will save at the same path if not 

In [15]:
print(data.__doc__)


    Class to handle splitting, normalizing, encoding ... data to input to the models.
    
